In [ ]:
!pip install pypulseq==1.3.1.post1 &> /dev/null
!pip install MRzeroCore &> /dev/null
!wget https://github.com/MRsources/MRzero-Core/raw/main/documentation/playground_mr0/numerical_brain_cropped.mat &> /dev/null

(GRE2FLASH_seq)=
# Interactive GRE to FLASH

In [ ]:
#@title Define GRE in PyPulseq
#@markdown You can steer different settings by the sliders, like long TR spoiling, gradient spoiling or rf spoiling. You can investigate artifacts that occur for short TR, and can be eliminated with these measures. Check https://mriquestions.com/spoiling---what-and-how.html
import numpy as np
# newer numpy versions don't contain this, but pypulseq still relies on it
np.int = int
np.float = float
np.complex = complex

import MRzeroCore as mr0
import pypulseq as pp
import torch
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

experiment_id = 'exB05_FLASH_2D'

# %% S1. SETUP sys

# choose the scanner limits
system = pp.Opts(max_grad=28,grad_unit='mT/m',max_slew=150,slew_unit='T/m/s',
                 rf_ringdown_time=20e-6,rf_dead_time=100e-6,adc_dead_time=20e-6,grad_raster_time=50*10e-6)

# %% S2. DEFINE the sequence
seq = pp.Sequence()

# Define FOV and resolution
fov = 200e-3
slice_thickness = 8e-3
sz = (32, 32)   # spin system size / resolution
Nread = 64    # frequency encoding steps/samples
Nphase = 64    # phase encoding steps/samples


# rf1 = pp.make_block_pulse(flip_angle=90 * np.pi / 180, duration=1e-3, system=system)

# Define other gradients and ADC events
gx = pp.make_trapezoid(channel='y', flat_area=Nread / fov, flat_time=10e-3, system=system)
adc = pp.make_adc(num_samples=Nread, duration=10e-3, phase_offset=0 * np.pi/180, delay=gx.rise_time, system=system)
gx_pre = pp.make_trapezoid(channel='y', area=-gx.area / 2, duration=5e-3, system=system)
gx_spoil = pp.make_trapezoid(channel='y', area=1.5 * gx.area, duration=2e-3, system=system)

rf_phase = 0
rf_inc = 0
rf_spoiling_inc = 117

P_rfspoil=True #@param {type: "boolean"}
P_gspoil=True  #@param {type: "boolean"}
P_TR_delay=0   #@param {type: "number"}
P_FA=10         #@param {type: "slider", min: 1, max: 90}
P_PE=True #@param {type: "boolean"}
P_PE_rewinder=True #@param {type: "boolean"}

# Define rf events
rf1, _, _ = pp.make_sinc_pulse(
    flip_angle=P_FA * np.pi / 180, duration=1e-3,slice_thickness=slice_thickness, apodization=0.5, time_bw_product=4,
    system=system, return_gz=True
)

# ======
# CONSTRUCT SEQUENCE
# ======
##linear reordering
phenc = np.arange(-Nphase // 2, Nphase // 2, 1) / fov
permvec =np.arange(0, Nphase, 1)
## centric reordering
permvec = sorted(np.arange(len(phenc)), key=lambda x: abs(len(phenc) // 2 - x))
## random reordering
#perm =np.arange(0, Nphase, 1);  permvec = np.random.permutation(perm)

phenc_centr = phenc[permvec] *P_PE

for ii in range(0, Nphase):  # e.g. -64:63

    if P_rfspoil:
      rf1.phase_offset = rf_phase / 180 * np.pi   # set current rf phase
      adc.phase_offset = rf_phase / 180 * np.pi  # follow with ADC
      rf_inc = divmod(rf_inc + rf_spoiling_inc, 360.0)[1]   # increase increment
      # increment additional pahse
      rf_phase = divmod(rf_phase + rf_inc, 360.0)[1]


    seq.add_block(rf1)
    seq.add_block(pp.make_delay(0.005))
    gp = pp.make_trapezoid(channel='x', area=phenc_centr[ii], duration=5e-3, system=system)
    seq.add_block(gx_pre, gp)
    seq.add_block(adc, gx)
    gp = pp.make_trapezoid(channel='x', area=-phenc_centr[ii], duration=5e-3, system=system)
    if P_gspoil and P_PE_rewinder:
      seq.add_block(gx_spoil, gp)
    elif P_gspoil:
      seq.add_block(gx_spoil)
    elif P_PE_rewinder:
      seq.add_block(gp)
    if ii < Nphase - 1:
        seq.add_block(pp.make_delay(P_TR_delay))


# %% S3. CHECK, PLOT and WRITE the sequence  as .seq
# Check whether the timing of the sequence is correct
ok, error_report = seq.check_timing()
if ok:
    print('Timing check passed successfully')
else:
    print('Timing check failed. Error listing follows:')
    [print(e) for e in error_report]

# PLOT sequence
#sp_adc, t_adc = mr0.util.pulseq_plot(seq, clear=False, figid=(11,12))

# Prepare the sequence output for the scanner
seq.set_definition('FOV', [fov, fov, slice_thickness])
seq.set_definition('Name', 'gre')
seq.write(experiment_id + '.seq')



In [ ]:
#@title Load Phantom and simulate


# %% S4: SETUP SPIN SYSTEM/object on which we can run the MR sequence external.seq from above
sz = [64, 64]
# (i) load a phantom object from file
obj_p = mr0.VoxelGridPhantom.load_mat('numerical_brain_cropped.mat')
obj_p = obj_p.interpolate(sz[0], sz[1], 1)
# Manipulate loaded data
obj_p.T2dash[:] = 30e-3
obj_p.D *= 0
obj_p.B0 *= 1    # alter the B0 inhomogeneity
# Store PD and B0 for comparison
PD = obj_p.PD
B0 = obj_p.B0
obj_p.plot()
# Convert Phantom into simulation data
obj_p = obj_p.build()


# %% S5:. SIMULATE  the external.seq file and add acquired signal to ADC plot
# Read in the sequence
seq0 = mr0.Sequence.import_file(experiment_id + '.seq')
#seq0.plot_kspace_trajectory()
# Simulate the sequence
graph = mr0.compute_graph(seq0, obj_p, 200, 1e-3)
signal = mr0.execute_graph(graph, seq0, obj_p, print_progress=False)

# PLOT sequence with signal in the ADC subplot
plt.close(11);plt.close(12)
sp_adc, t_adc = mr0.util.pulseq_plot(seq, clear=False, signal=signal.numpy())




In [ ]:
#@title Reconstruct and Plot


# additional noise as simulation is perfect
signal += 1e-4 * np.random.randn(signal.shape[0], 2).view(np.complex128)


# %% S6: MR IMAGE RECON of signal ::: #####################################
fig = plt.figure(figsize=(8, 5))  # fig.clf()
plt.subplot(411)
plt.title('ADC signal')
kspace = torch.reshape((signal), (Nphase, Nread)).clone()
plt.plot(torch.real(signal), label='real')
plt.plot(torch.imag(signal), label='imag')

ipermvec = np.argsort(permvec)

kspace=kspace[ipermvec,:]
# this adds ticks at the correct position szread
major_ticks = np.arange(0, Nphase * Nread, Nread)
ax = plt.gca()
ax.set_xticks(major_ticks)
for label in ax.xaxis.get_ticklabels():
    label.set_visible(False)
plt.grid()

# fftshift
spectrum = torch.fft.fftshift(kspace)
# FFT
space = torch.fft.fft2(spectrum)
# fftshift
space = torch.fft.ifftshift(space)


plt.subplot(345)
plt.title('k-space')
plt.imshow(np.abs(kspace.numpy().T), origin="lower"); plt.axis('off');plt.colorbar()
plt.subplot(349)
plt.title('log. k-space')
plt.imshow(np.log(np.abs(kspace.numpy().T)), origin="lower"); plt.axis('off');plt.colorbar()

plt.subplot(346)
plt.title('FFT-mag.')
plt.imshow(np.abs(space.numpy().T), origin="lower"); plt.axis('off')
plt.colorbar()
plt.subplot(3, 4, 10)
plt.title('FFT-phase')
plt.imshow(np.angle(space.numpy().T), vmin=-np.pi, vmax=np.pi, origin="lower"); plt.axis('off')
plt.colorbar()

# % compare with original phantom obj_p.PD
plt.subplot(348)
plt.title('phantom PD')
plt.imshow(PD.squeeze().T, origin="lower"); plt.axis('off');plt.colorbar()
plt.subplot(3, 4, 12)
plt.title('phantom B0')
plt.imshow(B0.squeeze().T, origin="lower"); plt.axis('off');plt.colorbar()